In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np
from room_simulation import Simulation
from sine_dataset import SineData
from network import SSLConvNet as ConvNet
from network import SimpleNet as SimpleNet
from logger import Logger
import matplotlib.pyplot as plt
import math
import seaborn as sns
import gc

In [ ]:
# Important paramters for training DNN
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
print(device)
EPOCHS = 7000
BATCH_SIZE = 32
STEP_SIZE = 10

# Important simulation and data constants
RAD = 50                            # Radius of ellipse of microphone placements in cm
RADII = [50, 500, 5000]             # Radii that will be tested in experiment 2
ROOM_SIMS = []                      
DATASETS = []
MIC_L_DIST = (11, -10)              # Distance between center and left microphone in cm on x-axis
MIC_R_DIST = (11, -10)              # Distance between center and right microphone in cm on y-axis
ABSORPTION = 0.0                    # Absorption factor of the room in the simulation (1 = no reverb)
MIN_FREQ = 20
MAX_FREQ = 20000                     # Minimal frequency of a sine wave according to Nyquist theorem
SAMPLE_RATE = int(MAX_FREQ*2.2)     # Sampling rate of signals in Hz
TIME = 1                            # Time in seconds of sine wave
MIN_LENGTH_EXP2 = 65000             # Length a single sine signal should have
MIN_LENGTH_EXP1 = 48000             # Length a single sine signal should have
TO_RAD = np.pi/180                  # Constant to convert degrees to radians
TO_DEG = 180/np.pi                  # Constant to convert radians to degrees

for rad in RADII:
    room_sim = Simulation(SAMPLE_RATE, rad, ABSORPTION, MIC_L_DIST, MIC_R_DIST)
    ROOM_SIMS.append(room_sim)

    DATASETS.append(SineData(BATCH_SIZE, room_sim, TIME, MIN_LENGTH_EXP2, MIN_FREQ, MAX_FREQ))

In [ ]:
# Plot room
room_sim.plot_room()

# Define custom loss function
class CosBorderLoss(torch.nn.Module):
    def __init__(self):
        super(CosBorderLoss, self).__init__()

    def forward(self, pred, target):
        radial = torch.abs(torch.cos(pred-target) - torch.cos(target-target))
        border = nn.functional.relu(pred-2*np.pi) + nn.functional.relu(-pred)
        return torch.sum(radial + border)

# Plot heatmap of custom loss function
cos_loss = CosBorderLoss()
cos_border_loss = np.zeros((1080,1080))
i, j = 0, 0
for i,deg1 in enumerate(np.arange(-360,720)):
    for j,deg2 in enumerate(np.arange(-360, 720)):
        rad1 = torch.as_tensor(deg1*TO_RAD)
        rad2 = torch.as_tensor(deg2*TO_RAD)
        cos_border_loss[i, j] = cos_loss(rad1, rad2)
ax = sns.heatmap(cos_border_loss)
plt.title("Heatmap of loss function based on cos + border")
plt.xlabel("Angles in degrees (should be range [-2pi,4pi])")
plt.ylabel("Angles in degrees (should be range [-2pi,4pi])")
plt.show()

In [ ]:
# TRAIN NETWORK FOR EXPERIMENT 1

# Initialize network and train parameters
if EPOCHS > 0:
    for i in range(len(RADII)):
        print("----------- MODEL " + str(i) + " -----------")
        print("Rad = "+str(RADII[i]))
        net = ConvNet(DATASETS[i].MIN_LENGTH)
        net = net.double().to(device)
        mse_loss = nn.MSELoss()
        optimiser = torch.optim.Adam(net.parameters())
        logger = Logger("./logs", "RAD" + str(RADII[i]))
        dataloader = DataLoader(DATASETS[i], batch_size=BATCH_SIZE)

        for curEpoch in range(EPOCHS):
            # Reset seed so random sine data is generated, otherwise RNG is repeated
            np.random.seed()

            # Gather input data & labels
            inL, inR, labelX, labelY, _ = next(iter(dataloader))
            inL = inL.double().to(device)
            inR = inR.double().to(device)
            labelX = labelX.double().to(device)
            labelY = labelY.double().to(device)

            # Training loop
            optimiser.zero_grad()
            outputX, outputY = net(inL, inR)
            outputX = torch.squeeze(outputX)
            outputY = torch.squeeze(outputY)
            loss = mse_loss(outputX, labelX) + mse_loss(outputY, labelY)
            loss.backward()
            optimiser.step()

            # Tensorboard and plotting
            if curEpoch % STEP_SIZE == 0: 
                # Log loss in tensorboard as scalars
                logger.scalar_summary("loss", loss.item(), curEpoch)

                outputXnp = outputX.detach().cpu().numpy()
                outputYnp = outputY.detach().cpu().numpy()
                labelXnp = labelX.detach().cpu().numpy()
                labelYnp = labelY.detach().cpu().numpy()

                # Plot the predicted vs actual angles
                if curEpoch % 100 == 0:
                    print("Loss of epoch "+str(curEpoch)+" = "+str(loss.item()))
                    
                    plt.scatter(labelXnp, outputXnp, c="red", label = "X")
                    plt.scatter(labelYnp, outputYnp, c="blue", label = "Y")
                    plt.title("Pred X (red) and Y (blue) coords vs actual coords of epoch " + str(curEpoch) + " of radius = " + str(RADII[i]))
                    plt.xlabel("Actual X/Y values")
                    plt.ylabel("Predicted X/Y values")
                    plt.legend()
                    plt.show()

                # Log gradients and weights in tensorboard as distributions and histograms
                for tag, value in net.named_parameters():
                    # Log gradient of last layer as scalar
                    if "fc" in tag and "weight" in tag:
                        last_grad_norm = np.sqrt(np.mean(np.square(value.grad.data.cpu().numpy())))
                        logger.scalar_summary("last_grad_norm", last_grad_norm, curEpoch)

            curEpoch += 1

        del net, inL, inR, labelX, labelY
        gc.collect()
            
        torch.save(net.state_dict(), "ConvNet_Rad" + str(RADII[i]))

In [ ]:
# TRAIN NETWORK FOR EXPERIMENT 2

room_sim = Simulation(SAMPLE_RATE, RAD, ABSORPTION, MIC_L_DIST, MIC_R_DIST)
dataset = SineData(BATCH_SIZE, room_sim, TIME, MIN_LENGTH_EXP2, MIN_FREQ, MAX_FREQ)

if EPOCHS > 0:
    net = ConvNet(MIN_LENGTH_EXP2)
    net = net.double().to(device)
    mse_loss = nn.MSELoss()
    optimiser = torch.optim.Adam(net.parameters())
    logger = Logger("./logs", "MSE")
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE)

    for curEpoch in range(EPOCHS):
        # Reset seed so random sine data is generated, otherwise RNG is repeated
        np.random.seed()

        # Gather input data & labels
        inL, inR, labelX, labelY, _ = next(iter(dataloader))
        inL = inL.double().to(device)
        inR = inR.double().to(device)
        labelX = labelX.double().to(device)
        labelY = labelY.double().to(device)

        # Training loop
        optimiser.zero_grad()
        outputX, outputY = net(inL, inR)
        loss = mse_loss(torch.squeeze(outputX), labelX) + mse_loss(torch.squeeze(outputY), labelY)
        loss.backward()
        optimiser.step()

        # Tensorboard and plotting
        if curEpoch % STEP_SIZE == 0: 
            # Log loss in tensorboard as scalars
            logger.scalar_summary("loss", loss.item(), curEpoch)

            outputXnp = outputX.detach().cpu().numpy()
            outputYnp = outputY.detach().cpu().numpy()
            labelXnp = labelX.detach().cpu().numpy()
            labelYnp = labelY.detach().cpu().numpy()

            # Plot the predicted vs actual angles
            if curEpoch % 100 == 0:
                print("Loss of epoch "+str(curEpoch)+" = "+str(loss.item()))
                
                plt.scatter(labelXnp, outputXnp, c="red", label = "X")
                plt.scatter(labelYnp, outputYnp, c="blue", label = "Y")
                plt.title("Pred X (red) and Y (blue) coords vs actual coords of epoch " + str(curEpoch) + " of radius = " + str(RADII[i]))
                plt.xlabel("Actual X/Y values")
                plt.ylabel("Predicted X/Y values")
                plt.legend()
                plt.show()
                
                # Log gradients and weights in tensorboard as distributions and histograms
                for tag, value in net.named_parameters():
                    # Log gradient of last layer as scalar
                    if "fc" in tag and "weight" in tag:
                        last_grad_norm = np.sqrt(np.mean(np.square(value.grad.data.cpu().numpy())))
                        logger.scalar_summary("last_grad_norm", last_grad_norm, curEpoch)

        curEpoch += 1

    torch.save(net.state_dict(), "ConvNet_CosLoss")

In [ ]:
# TRAIN BASELINE MODEL

room_sim = Simulation(SAMPLE_RATE, RAD, ABSORPTION, MIC_L_DIST, MIC_R_DIST)
dataset = SineData(BATCH_SIZE, room_sim, TIME, MIN_LENGTH_EXP1, MIN_FREQ, MAX_FREQ)

if EPOCHS > 0:
    net = SimpleNet(MIN_LENGTH_EXP1)
    net = net.double().to(device)
    mse_loss = nn.MSELoss()
    optimiser = torch.optim.Adam(net.parameters())
    logger = Logger("./logs", "MSE")
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE)

    for curEpoch in range(EPOCHS):
        # Reset seed so random sine data is generated, otherwise RNG is repeated
        np.random.seed()

        # Gather input data & labels
        inL, inR, labelX, labelY, _ = next(iter(dataloader))
        inL = inL.double().to(device)
        inR = inR.double().to(device)
        labelX = labelX.double().to(device)
        labelY = labelY.double().to(device)

        # Training loop
        optimiser.zero_grad()
        outputX, outputY = net(inL, inR)
        loss = mse_loss(torch.squeeze(outputX), labelX) + mse_loss(torch.squeeze(outputY), labelY)
        loss.backward()
        optimiser.step()

        # Tensorboard and plotting
        if curEpoch % STEP_SIZE == 0: 
            # Log loss in tensorboard as scalars
            logger.scalar_summary("loss", loss.item(), curEpoch)

            outputXnp = outputX.detach().cpu().numpy()
            outputYnp = outputY.detach().cpu().numpy()
            labelXnp = labelX.detach().cpu().numpy()
            labelYnp = labelY.detach().cpu().numpy()

            # Plot the predicted vs actual angles
            if curEpoch % 100 == 0:
                print("Loss of epoch "+str(curEpoch)+" = "+str(loss.item()))
                
                plt.scatter(labelXnp, outputXnp, c="red", label = "X")
                plt.scatter(labelYnp, outputYnp, c="blue", label = "Y")
                plt.title("Pred X (red) and Y (blue) coords vs actual coords of epoch " + str(curEpoch) + " of radius = " + str(RAD))
                plt.xlabel("Actual X/Y values")
                plt.ylabel("Predicted X/Y values")
                plt.legend()
                plt.show()

                # Log gradients and weights in tensorboard as distributions and histograms
                for tag, value in net.named_parameters():
                    # Log gradient of last layer as scalar
                    if "fc" in tag and "weight" in tag:
                        last_grad_norm = np.sqrt(np.mean(np.square(value.grad.data.cpu().numpy())))
                        logger.scalar_summary("last_grad_norm", last_grad_norm, curEpoch)

        curEpoch += 1

    torch.save(net.state_dict(), "SimpleNet_MSELoss")